In [ ]:
# Gravar as versões de outros pacotes usados
!pip install -q -U watermark

In [ ]:
# Instalar pacote iPython-sql para possibilitar a utilização de queries sql dentro do notebook
!pip install -q ipython-sql
!pip install --upgrade --user bottleneck

In [ ]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Criamos a conexão a um banco de dados SQLite
cnn = sqlite3.connect('database/dbprojeto1.db')

In [ ]:
# Carregamos a extensão SQL
%load_ext sql

In [ ]:
# Definimos o banco de dados
%sql sqlite:///database/dbprojeto1.db

# Pima Indians Diabetes Database | Estudo de caso

Precisamos gerar uma amostra de dados com os pacientes com mais de 50 anos e para cada um deles indicar em uma nova coluna se o paciente está normal (BMI menor que 30) ou obeso (BMI maior ou igual a 30). Então devemos gerar um novo arquivo CSV e encaminhar para o Cientista de Dados.

In [ ]:
df = pd.read_csv('dataset/diabetes.csv')

In [ ]:
df.head()

In [ ]:
# Copia o dataframe para dentro do banco de dados como uma tabela
df.to_sql('diabetes', cnn)

In [ ]:
%%sql

SELECT count(*) FROM diabetes

In [ ]:
%%sql

SELECT Age, Glucose, Outcome FROM diabetes WHERE Glucose > 190

In [ ]:
df.columns

In [ ]:
%%sql

CREATE TABLE pacientes (Pregnancies INT,
                        Glucose INT,
                        BloodPressure INT,
                        SkinThickness INT,
                        Insulin INT,
                        BMI DECIMAL(8, 2),
                        DiabetesPedigreeFunction DECIMAL(8, 2),
                        Age INT,
                        Outcome INT);

In [ ]:
%%sql

SELECT * FROM pacientes

In [ ]:
%%sql

INSERT INTO pacientes(Pregnancies, 
                      Glucose, 
                      BloodPressure, 
                      SkinThickness, 
                      Insulin, 
                      BMI, 
                      DiabetesPedigreeFunction, 
                      Age, 
                      Outcome) 
SELECT Pregnancies, 
       Glucose, 
       BloodPressure, 
       SkinThickness, 
       Insulin, 
       BMI, 
       DiabetesPedigreeFunction, 
       Age, 
       Outcome 
FROM diabetes WHERE Age > 50;

In [ ]:
%%sql

SELECT * FROM pacientes

In [ ]:
%%sql 

ALTER TABLE pacientes
ADD Perfil VARCHAR(10);

In [ ]:
%%sql

SELECT * FROM pacientes

In [ ]:
%%sql

UPDATE pacientes
SET Perfil = 'Normal'
WHERE BMI < 30;

In [ ]:
%%sql

SELECT * FROM pacientes

In [ ]:
%%sql

UPDATE pacientes
SET Perfil = 'Obeso'
WHERE BMI >= 30;

In [ ]:
%%sql

SELECT * FROM pacientes

## Carregando os Dados no Pandas e Salvando o CSV

In [ ]:
# Query
query = cnn.execute("SELECT * FROM pacientes")
query

In [ ]:
# List Comprehension
cols = [coluna[0] for coluna in query.description]
cols

In [ ]:
# Gera o dataframe
resultado = pd.DataFrame.from_records(data = query.fetchall(), columns = cols)

In [ ]:
# Visualizar
resultado.head()

In [ ]:
# Salvar em CSV
resultado.to_csv('dataset/pacientes.csv', index = False)

In [ ]:
pacientes_df = pd.read_csv('dataset/pacientes.csv')
# Filtrar pacientes com mais de 50 anos
pacientes_maior_50 = pacientes_df[pacientes_df['Age'] > 50]

# Contagem de pacientes por perfil
contagem_perfil = pacientes_maior_50['Perfil'].value_counts()

# Gráfico representativo
plt.figure(figsize=(8, 6))
sns.barplot(x=contagem_perfil.index, y=contagem_perfil.values, palette=['red', 'green'])
plt.title('Distribuição de Pacientes Normais e Obesos com Mais de 50 Anos')
plt.xlabel('Perfil')
plt.ylabel('Número de Pacientes')
for index, value in enumerate(contagem_perfil.values):
    plt.text(index, value + 1, str(value), ha='center')

# Exibir o gráfico
plt.show()

# Observações:

## Número de Pacientes Normais e Obesos:

* A barra verde representa o número de pacientes considerados normais.
* A barra vermelha representa o número de pacientes obesos.
* Podemos observar que há uma diferença significativa no número de pacientes entre os dois perfis.